In [9]:
'''Make possible for python notebooks to import the util as module. This needs to be copied at every python notebook which wants to 
load the module'''

import io, os, sys, types
from IPython import get_ipython
from nbformat import read
from IPython.core.interactiveshell import InteractiveShell

def find_notebook(fullname, path=None):
    """find a notebook, given its fully qualified name and an optional path
    
    This turns "foo.bar" into "foo/bar.ipynb"
    and tries turning "Foo_Bar" into "Foo Bar" if Foo_Bar
    does not exist.
    """
    name = fullname.rsplit('.', 1)[-1]
    if not path:
        path = ['']
    for d in path:
        nb_path = os.path.join(d, name + ".ipynb")
        if os.path.isfile(nb_path):
            return nb_path
        # let import Notebook_Name find "Notebook Name.ipynb"
        nb_path = nb_path.replace("_", " ")
        if os.path.isfile(nb_path):
            return nb_path


class NotebookLoader(object):
    """Module Loader for Jupyter Notebooks"""
    def __init__(self, path=None):
        self.shell = InteractiveShell.instance()
        self.path = path
    
    def load_module(self, fullname):
        """import a notebook as a module"""
        path = find_notebook(fullname, self.path)
        
        print ("importing Jupyter notebook from %s" % path)
                                       
        # load the notebook object
        with io.open(path, 'r', encoding='utf-8') as f:
            nb = read(f, 4)

        # create the module and add it to sys.modules
        # if name in sys.modules:
        #    return sys.modules[name]
        mod = types.ModuleType(fullname)
        mod.__file__ = path
        mod.__loader__ = self
        mod.__dict__['get_ipython'] = get_ipython
        sys.modules[fullname] = mod
        
        # extra work to ensure that magics that would affect the user_ns
        # actually affect the notebook module's ns
        save_user_ns = self.shell.user_ns
        self.shell.user_ns = mod.__dict__
        
        try:
          for cell in nb.cells:
            if cell.cell_type == 'code':
                # transform the input to executable Python
                code = self.shell.input_transformer_manager.transform_cell(cell.source)
                # run the code in themodule
                exec(code, mod.__dict__)
        finally:
            self.shell.user_ns = save_user_ns
        return mod


class NotebookFinder(object):
    """Module finder that locates Jupyter Notebooks"""
    def __init__(self):
        self.loaders = {}
    
    def find_module(self, fullname, path=None):
        nb_path = find_notebook(fullname, path)
        if not nb_path:
            return
        
        key = path
        if path:
            # lists aren't hashable
            key = os.path.sep.join(path)
        
        if key not in self.loaders:
            self.loaders[key] = NotebookLoader(path)
        return self.loaders[key]
    
sys.meta_path.append(NotebookFinder())

In [10]:
import tm_assignment_util as util
myutilObj = util.util()

In [11]:
#Let's Build the Model
# Apply preprocessing to every document in the training set
X_Toks_Trn = util.X_Cases_Trn.apply(myutilObj.my_tokens_as_text)
X_Toks_Tst = util.X_Cases_Tst.apply(myutilObj.my_tokens_as_text)

In [12]:
#Naive Bayes
#Build a pipeline: Combine multiple steps into one
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer(use_idf=True)),
                      ('clf', MultinomialNB()),
                    ])

In [13]:
text_clf.fit(X_Toks_Trn, util.Y_Cases_Trn)

Pipeline(steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...inear_tf=False, use_idf=True)), ('clf', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [14]:
util.Y_Cases_Trn.describe()

count      2894
unique       12
top       Falls
freq       1045
Name: Cause, dtype: object

In [15]:
predicted = text_clf.predict(X_Toks_Tst)

In [16]:
#Test model accuracy
import numpy as np
from sklearn import metrics
print(metrics.confusion_matrix(util.Y_Cases_Tst, predicted))
print(np.mean(predicted == util.Y_Cases_Tst))
#y_test.value_counts()
print(metrics.classification_report(util.Y_Cases_Tst, predicted))

[[ 28  12   0   0   0   0  88   0   0   1   0]
 [  0  40   0   0   0   0  92   0   0   0   0]
 [  0   0   0   0   0   0  17   0   0   0   0]
 [  0   0   0  27   0   0   8   0   0   0   0]
 [  1   1   0   0   1   0  27   0   0   0   0]
 [  0   0   0   1   0   0  19   3   0   0   0]
 [  0   0   0   0   0   0 413   0   0   0   0]
 [  0   2   0   4   0   0  26   4   0   0   0]
 [  0   2   0   0   0   0   7   0   0   0   0]
 [  4   2   0   0   0   0  38   0   0   9   0]
 [  3   3   0   0   0   0  23   0   0   0   0]]
0.576158940397
                                  precision    recall  f1-score   support

       Caught in/between Objects       0.78      0.22      0.34       129
              Collapse of object       0.65      0.30      0.41       132
                        Drowning       0.00      0.00      0.00        17
                   Electrocution       0.84      0.77      0.81        35
 Exposure to Chemical Substances       1.00      0.03      0.06        30
Exposure to extreme te

C:\Users\ranjanm\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [17]:
predicted_target_nb = text_clf.predict(util.accidentCases_Osha.Title_Summary_Case.apply(myutilObj.my_tokens_as_text))

In [18]:
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt

Counter(predicted_target_nb)

Counter({'Caught in/between Objects': 1348,
         'Collapse of object': 249,
         'Electrocution': 791,
         'Exposure to Chemical Substances': 11,
         'Falls': 10184,
         'Fires and Explosion': 213,
         'Struck By Moving Objects': 34})